# Comparison of SHRED models with different sensing strategies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

path_svd = './OfflineSVD/'
idx_params, params, var_names, is_vector, fom_times, rescaling_snaps, Nmodes = pickle.load(open(path_svd+'msfr_p.uloff', 'rb'))

Let us recompute the scalers for the POD coefficients (better to implement a way to import them)

In [2]:
import pickle
from sklearn.preprocessing import MinMaxScaler

v_POD = pickle.load(open(path_svd+'v_POD.svd', 'rb'))

assert sum(Nmodes) == v_POD['train'].shape[2]

vpod_scaler = MinMaxScaler()
vpod_scaler.fit(v_POD['train'].reshape(-1, sum(Nmodes)))

rescaled_v_POD = {
    key: vpod_scaler.transform(v_POD[key].reshape(-1, sum(Nmodes))).reshape(v_POD[key].shape)
    for key in list(v_POD.keys())
}

Let us load the test datasets

In [3]:
import sys
sys.path.append('../../')

from shred.processdata import Padding, TimeSeriesDataset
import torch

path_test_datasets = {
    'Out-Core': 'Test_results/',
    'Mobile Sensors': 'Test_results/MobileSensor/',    
    'Mobile Probes': 'Test_results/MobileProbes/'
}

test_datasets = {
    key: pickle.load(open(path_test_datasets[key]+'test_dataset.pkl', 'rb'))
    for key in path_test_datasets.keys()
}

# They are all loaded, but the .Y of each time serie is actually the same!!!

n_configurations = {
    key: len(test_datasets[key])
    for key in path_test_datasets.keys()
}

input_size = {
    key: test_datasets[key][0].X.shape[-1]
    for key in path_test_datasets.keys()
}

output_size = {
    key: test_datasets[key][0].Y.shape[-1]
    for key in path_test_datasets.keys()
}

print(n_configurations)
print(input_size)
print(output_size)

{'Out-Core': 10, 'Mobile Sensors': 5, 'Mobile Probes': 4}
{'Out-Core': 3, 'Mobile Sensors': 3, 'Mobile Probes': 6}
{'Out-Core': 221, 'Mobile Sensors': 221, 'Mobile Probes': 221}


Let us load the SHRED models

In [6]:
path_shreds = {
    'Out-Core': 'SHRED/',
    'Mobile Sensors': 'SHRED/MobileSensor/',
    'Mobile Probes': 'SHRED/MobileProbes/',
}

device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')

measured_field = 3

from shred.models import SHRED
shred_models = dict()

for key in path_shreds.keys():
    shred_models[key] = list()
    for kk in range(n_configurations[key]):
        shred_models[key].append(
            SHRED( input_size[key], output_size[key], 
            hidden_size = 64, hidden_layers = 2, decoder_sizes = [350, 400], dropout = 0.1).to(device)
            )
        
        if key == 'Out-Core':
            path_shred_mod = path_shreds[key]+'trained_config'+str(kk)+'_measuring_'+str(input_size[key])+'sens_'+var_names[measured_field]+'.shred'
        elif key == 'Mobile Sensors':
            path_shred_mod = path_shreds[key]+'trained_config'+str(kk)+'_measuring_'+str(input_size[key]/2)+'probes.shred'
        elif key == 'Mobile Probes':
            path_shred_mod = path_shreds[key]+'trained_config'+str(kk)+'_measuring_'+str(input_size[key])+'probes.shred'
        
        shred_models[key][kk].load_state_dict(
            torch.load(path_shred_mod,map_location=device)
            )
        
        shred_models[key][kk].freeze()

Let us evaluate the output of the SHRED models

In [7]:
from shred.processdata import num2p, mre

reshaped_Ytest_hat = dict()

for key in path_shreds.keys():
    Ytest_POD_hat = torch.stack([shred_models[key][kk](test_datasets[key][kk].X) for kk in range(n_configurations[key])], dim=0)

    Ytest_POD_pred = {
        'mean': Ytest_POD_hat.mean(axis=0),
        'std':  Ytest_POD_hat.std(axis=0) / np.sqrt(n_configurations[key])
    }

    # The test data are independent on the configurations of the sensors
    print(key)
    print("Mean relative SHRED prediction error on POD coeffs: %s." % num2p(mre(test_datasets[key][-1].Y, Ytest_POD_pred['mean'])))
    print("Std  relative SHRED prediction error on POD coeffs: %s." % num2p((Ytest_POD_pred['std'].pow(2).sum(axis = -1).sqrt() / (test_datasets[key][-1].Y).pow(2).sum(axis = -1).sqrt()).mean()))
    print('-------------------------')

    reshaped_Ytest_hat[key] = {
                            'mean': Ytest_POD_pred['mean'].cpu().detach().numpy().reshape(len(idx_params['test']), len(fom_times), output_size[key]),
                            'std':  Ytest_POD_pred['std'].cpu().detach().numpy().reshape(len(idx_params['test']), len(fom_times), output_size[key])
                         }
    
reshaped_Ytest = test_datasets['Out-Core'][-1].Y.cpu().detach().numpy().reshape(len(idx_params['test']), len(fom_times), output_size['Out-Core'])

Out-Core
Mean relative SHRED prediction error on POD coeffs: 3.00%.
Std  relative SHRED prediction error on POD coeffs: 0.90%.
-------------------------
Mobile Sensors
Mean relative SHRED prediction error on POD coeffs: 3.96%.
Std  relative SHRED prediction error on POD coeffs: 2.17%.
-------------------------
Mobile Probes
Mean relative SHRED prediction error on POD coeffs: 7.23%.
Std  relative SHRED prediction error on POD coeffs: 3.40%.
-------------------------


Let us decode to the high-dimensional space and compute the relative reconstruction error

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output as clc

path_snaps = '../../NuSHRED_Datasets/D2/'

compute_errors = True
if compute_errors:
    ave_rel_errors = dict()

    for key in path_shreds.keys():

        print('Computing errors for '+key)

        ave_rel_errors[key] = np.zeros((len(var_names), len(idx_params['test']), len(fom_times)))

        for field_i, field in enumerate(var_names):
            idx_to_rec = np.arange(sum(Nmodes[:field_i]),  sum(Nmodes[:field_i+1]),  1, dtype=int)

            # Load compressed dataset
            u_data  = pickle.load(open(path_snaps + f'CompressedDataset/pod_basis_{field}.svd', 'rb'))
            s_data  = pickle.load(open(path_snaps + f'CompressedDataset/sing_vals_{field}.svd', 'rb'))
            vh_data = pickle.load(open(path_snaps + f'CompressedDataset/v_POD_all_fields.svd', 'rb'))[field]

            for param_to_recon in tqdm(range(len(idx_params['test'])), 'Computing error for '+field):
                fom = u_data @ s_data @ vh_data[idx_params['test'][param_to_recon]].T

                u_svd = pickle.load(open(path_svd+'pod_basis_'+field+'.svd', 'rb'))
                s_svd = pickle.load(open(path_svd+'sing_vals_'+field+'.svd', 'rb'))

                _tmp_mean_v = vpod_scaler.inverse_transform(reshaped_Ytest_hat[key]['mean'][:,:,:-1].reshape(-1, sum(Nmodes))).reshape(reshaped_Ytest_hat[key]['mean'][:,:,:-1].shape)
                recon     = u_svd @ np.diag(s_svd) @ _tmp_mean_v[param_to_recon, :, idx_to_rec]
        
                ave_rel_errors[key][field_i, param_to_recon, :] = np.linalg.norm(fom - recon, axis = 0) / np.linalg.norm(fom, axis = 0)

                del fom, recon
        clc()
        
    # Save the average relative errors
    pickle.dump(ave_rel_errors, open('Test_results/comparison_ave_rel_errors.pkl', 'wb'))
else:
    ave_rel_errors = pickle.load(open(path_test_datasets['Out-Core']+'ave_rel_errors.pkl', 'rb'))

Computing errors for Out-Core


Computing error for U:   0%|          | 0/3 [00:00<?, ?it/s]

Let us define latex variables

In [17]:
tex_var_names = [r'\mathbf{u}', 'T', '\Phi']
energy_groups = 6
tex_var_names.extend([r'\phi_'+str(g+1) for g in range(energy_groups)])

prec_groups = 8
tex_var_names.extend([r'c_'+str(g+1) for g in range(prec_groups)])

tex_var_names.extend(['p', r'p_{\text{rgh}}', r'\kappa', r'\nu_t', "q'''"])

assert len(tex_var_names) == len(var_names)

Let us create a table for the errors

In [19]:
import pandas as pd

df = pd.DataFrame(columns=ave_rel_errors.keys(), index=tex_var_names)

for key in ave_rel_errors.keys():
    for field_i, field in enumerate(tex_var_names):
        df.loc[field, key] = ave_rel_errors[key][field_i].mean()

df

,Out-Core,Mobile Sensors,Mobile Probes
\mathbf{u},0.001563,0.008265,0.004867
T,0.012784,0.032119,0.022397
\Phi,0.00479,0.065088,0.046097
\phi_1,0.004775,0.065345,0.046468
\phi_2,0.004774,0.065096,0.045981
\phi_3,0.004783,0.06517,0.046364
\phi_4,0.004786,0.064832,0.04614
\phi_5,0.004781,0.065322,0.046122
\phi_6,0.004785,0.065552,0.046094
c_1,0.015392,0.048613,0.061521
